In [117]:
from pprint import pprint
import pandas as pd
import yfinance as yf
from tqdm import tqdm

In [118]:


#https://www.b3.com.br/pt_br/market-data-e-indices/servicos-de-dados/market-data/consultas/boletim-diario/dados-publicos-de-produtos-listados-e-de-balcao/
# Usamos apenas as colunas em que precisamos aplicar filtros
# O arquivo .csv não vem formatado em utf-8, então usamos o encoding = 'latin'
# Definimos o dtype da coluna CorpGovnLvlNm como string para não termos problema com a memória
df = pd.read_csv(
    'InstrumentsConsolidatedFile_20231204_1.csv',
    usecols= ['RptDt','TckrSymb', 'SgmtNm', 'SctyCtgyNm', 'TradgStartDt', 'CorpGovnLvlNm'],
    sep=';',
    encoding = 'latin',
    dtype={'CorpGovnLvlNm': 'str'})

# Filtramos SctyCtgyNm por ações e units (SHARES e UNIT)
# Filtramos SgmtNm por ações do mercado à vista (CASH)
# Filtramos os ativos cuja data de início da negociação seja anterior à data de publicação do arquivo .csv
# Filtramos as ações que não sejam negociadas no mercado de balcão
df = df[ ( (df['SctyCtgyNm'] == 'SHARES') | (df['SctyCtgyNm'] == 'UNIT') ) & (df['SgmtNm'] == 'CASH') & (df['RptDt'] >= df['TradgStartDt']) & (df['CorpGovnLvlNm'] != 'MERCADO DE BALCÃO') ]

df_listed_stocks = pd.DataFrame()
df_listed_stocks['Ticker'] = df['TckrSymb']

# Ativos de nome TAXA100, TAXA101, TAXA102 etc. não nos interessam
df_listed_stocks = df_listed_stocks[~df_listed_stocks['Ticker'].str.contains('TAXA')]

df_listed_stocks = df_listed_stocks['Ticker'].reset_index()
df_listed_stocks = df_listed_stocks.drop(columns=['index'])

df_listed_stocks.to_csv('b3_stocks.csv', index=False)

print(df_listed_stocks['Ticker'])

0      AALR3
1      ABCB4
2      ABEV3
3      AERI3
4      AESB3
       ...  
549    WLMM4
550    YBRA3
551    YBRA4
552    YDUQ3
553    ZAMP3
Name: Ticker, Length: 554, dtype: object


In [119]:
stock_list = [stock[0] +'.SA' for stock in df_listed_stocks.values]

pprint(stock_list[:10])

['AALR3.SA',
 'ABCB4.SA',
 'ABEV3.SA',
 'AERI3.SA',
 'AESB3.SA',
 'AESO3.SA',
 'AFLT3.SA',
 'AGRO3.SA',
 'AGXY3.SA',
 'AHEB3.SA']


In [120]:
df.columns

Index(['RptDt', 'TckrSymb', 'SgmtNm', 'SctyCtgyNm', 'TradgStartDt',
       'CorpGovnLvlNm'],
      dtype='object')

In [121]:
start_date = '2022-01-01'
end_date = '2023-01-01'
df_cols = ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'ticker']
stocks_df = pd.DataFrame(columns=[df_cols])
stocks_df

,Open,High,Low,Close,Adj Close,Volume,ticker


In [122]:
for stock in tqdm(stock_list[:], desc="Processing"):
    print(stock)
    temp_df = yf.download(stock, start=start_date, end=end_date)
    if len(temp_df) != 0:
        temp_df = temp_df.assign(ticker = stock)
        stocks_df = pd.concat([stocks_df[df_cols], temp_df[df_cols]])

Processing:   0%|          | 0/554 [00:00<?, ?it/s]

AALR3.SA
[*********************100%%**********************]  1 of 1 completed
ABCB4.SA
[*********************100%%**********************]  1 of 1 completed
ABEV3.SA
[*********************100%%**********************]  1 of 1 completed
AERI3.SA
[*********************100%%**********************]  1 of 1 completed
AESB3.SA
[*********************100%%**********************]  1 of 1 completed
AESO3.SA
[*********************100%%**********************]  1 of 1 completed


1 Failed download:
['AESO3.SA']: Exception('%ticker%: No timezone found, symbol may be delisted')



AFLT3.SA
[*********************100%%**********************]  1 of 1 completed
AGRO3.SA
[*********************100%%**********************]  1 of 1 completed

Processing:   1%|▏         | 8/554 [00:00<00:07, 76.19it/s]

...


1 Failed download:
['YBRA4.SA']: Exception("%ticker%: Data doesn't exist for startDate = 1641006000, endDate = 1672542000")
Processing: 100%|█████████▉| 552/554 [04:40<00:01,  1.45it/s]


YDUQ3.SA
[*********************100%%**********************]  1 of 1 completed

Processing: 100%|█████████▉| 553/554 [04:40<00:00,  1.60it/s]


ZAMP3.SA
[*********************100%%**********************]  1 of 1 completed

Processing: 100%|██████████| 554/554 [04:40<00:00,  1.97it/s]

In [123]:
stocks_df['ticker'].unique()

array(['AALR3.SA', 'ABCB4.SA', 'ABEV3.SA', 'AERI3.SA', 'AESB3.SA',
       'AFLT3.SA', 'AGRO3.SA', 'AGXY3.SA', 'AHEB3.SA', 'AHEB5.SA',
       'AHEB6.SA', 'ALLD3.SA', 'ALPA3.SA', 'ALPA4.SA', 'ALPK3.SA',
       'ALUP11.SA', 'ALUP3.SA', 'ALUP4.SA', 'AMAR3.SA', 'AMBP3.SA',
       'AMER3.SA', 'ANIM3.SA', 'APER3.SA', 'ARML3.SA', 'ARZZ3.SA',
       'ASAI3.SA', 'ATMP3.SA', 'ATOM3.SA', 'AURE3.SA', 'AVLL3.SA',
       'AZEV3.SA', 'AZEV4.SA', 'AZUL4.SA', 'B3SA3.SA', 'BAHI3.SA',
       'BALM3.SA', 'BALM4.SA', 'BAUH3.SA', 'BAUH4.SA', 'BAZA3.SA',
       'BBAS3.SA', 'BBDC3.SA', 'BBDC4.SA', 'BBSE3.SA', 'BDLL3.SA',
       'BDLL4.SA', 'BEEF3.SA', 'BEES3.SA', 'BEES4.SA', 'BGIP3.SA',
       'BGIP4.SA', 'BHIA3.SA', 'BIOM3.SA', 'BLAU3.SA', 'BMEB3.SA',
       'BMEB4.SA', 'BMGB4.SA', 'BMIN3.SA', 'BMIN4.SA', 'BMKS3.SA',
       'BMOB3.SA', 'BNBR3.SA', 'BOBR3.SA', 'BOBR4.SA', 'BPAC11.SA',
       'BPAC3.SA', 'BPAC5.SA', 'BPAN4.SA', 'BRAP3.SA', 'BRAP4.SA',
       'BRBI11.SA', 'BRFS3.SA', 'BRGE11.SA', 'BRGE12.SA', 'B

In [124]:
stocks_df.to_parquet('b3_stock_data.parquet')

In [172]:
import backtrader as bt
import yfinance as yf
import matplotlib.pyplot as plt
import datetime
plot_w, plot_h = 15, 6

plt.style.use("default") #ggplot is also fine
plt.rcParams["figure.figsize"] = (plot_w,plot_h)

#all styles: print(pl.style.available)
%matplotlib inline

class BuyAndHoldStrategy(bt.Strategy):
    params = (
        ("allocation", [0.20, 0.20, 0.20, 0.15, 0.25]),
    )

    def __init__(self):
        self.stocks = self.datas

    def next(self):
        for i, d in enumerate(self.stocks):
            target_allocation = self.params.allocation[i] * self.broker.getvalue()
            current_allocation = self.broker.getposition(data=d).size * d.close[0]

            # Buy if the current allocation is less than the target allocation
            if current_allocation < target_allocation:
                amount_to_buy = target_allocation - current_allocation
                self.order_target_percent(data=d, target=self.params.allocation[i], exectype=bt.Order.Market)

cerebro = bt.Cerebro()

# List of stock symbols
stock_symbols = ['AZEV4.SA', 'AZUL4.SA', 'B3SA3.SA', 'VLID3.SA', 'VSPT3.SA']

#for symbol in stock_symbols:
#    data = bt.feeds.YahooFinanceData(dataname=symbol,
#                                        fromdate=datetime.datetime(2010, 1, 1),
#                                        todate=datetime.datetime(2023, 1, 1))
#    cerebro.adddata(data)

# Assuming you have a Pandas DataFrame 'df' with columns 'date', 'ticker', 'open', 'high', 'low', 'close', 'volume'
df = pd.read_csv('your_dataframe.csv')  # Replace 'your_dataframe.csv' with your actual CSV file

# Create a dictionary of DataFrames, one for each ticker
data = {ticker: bt.feeds.PandasData(dataname=df[df['ticker'] == ticker], name=ticker) for ticker in df['ticker'].unique()}

# Add data to Backtrader
for _, d in data.items():
    cerebro.adddata(d)

cerebro.addstrategy(BuyAndHoldStrategy)

cerebro.broker.set_cash(100000)
cerebro.broker.setcommission(commission=0.001)

# Print the starting cash amount
print(f"Starting Portfolio Value: {cerebro.broker.getvalue()}")

# Run the backtest
cerebro.run()

# Print the final cash amount
print(f"Ending Portfolio Value: {cerebro.broker.getvalue()}")

# Set a larger figure size for the candlestick plot
plt.figure(figsize=(plot_w, plot_h))

# Plot the portfolio value
cerebro.plot(style='candlestick')


Starting Portfolio Value: 100000


FileNotFoundError: [Errno 2] No such file or directory: 'AZEV4.SA'